# Introduction

This notebook was created to document the data and analysis on CWE (Common Weakness Enumeration). CWE represents the data in two different ways, a raw XML file and a structured hierarchy on the website. Below is the findings on the XML structure. 

# XML

The CWE table consists of a forest of 4 tables.

In [31]:
import lxml.etree
tree = lxml.etree.parse('cwec_v2.9.xml')
root = tree.getroot()
for table in root: 
    print (table.tag)

Views
Categories
Weaknesses
Compound_Elements


Below is a graphical representation of the XML structure.

In [32]:
from IPython.display import Image
Image(filename='CWE_Map.png')

The table we are concerned with is the **Weaknesses** table. I will discuss the others in a later commit. The first thing to observe is that the number of "columns" in every "row" of this table varies. For instance, let's consider row[0]:

In [33]:
weakness_table = root[2]
for row in weakness_table[0]: 
    print (row.tag)

Description
Relationships
Weakness_Ordinalities
Applicable_Platforms
Time_of_Introduction
Common_Consequences
Potential_Mitigations
Causal_Nature
Demonstrative_Examples
Taxonomy_Mappings
Content_History


And now let's consider row 20:

In [34]:
for row in weakness_table[20]: 
    print (row.tag)

Description
Relationships
Relationship_Notes
Weakness_Ordinalities
Applicable_Platforms
Alternate_Terms
Terminology_Notes
Time_of_Introduction
Likelihood_of_Exploit
Common_Consequences
Detection_Methods
Potential_Mitigations
Causal_Nature
Demonstrative_Examples
Observed_Examples
Functional_Areas
Affected_Resources
References
Taxonomy_Mappings
White_Box_Definitions
Related_Attack_Patterns
Content_History


Since we are interested in creating a histogram of all the fields used, we simple loop through every row of the table, and then loop through every column label counting them as they occur: 

In [35]:
histogram = {}
for row in weakness_table: 
    for column in row: 
        if column.tag not in histogram: 
            histogram[column.tag] = 0
        else:
            histogram[column.tag] += 1
print (histogram)

{'Causal_Nature': 74, 'Detection_Methods': 76, 'Common_Consequences': 701, 'Description': 718, 'Taxonomy_Mappings': 597, 'Other_Notes': 23, 'Modes_of_Introduction': 32, 'Maintenance_Notes': 86, 'Research_Gaps': 74, 'Weakness_Ordinalities': 130, 'Time_of_Introduction': 664, 'Content_History': 718, 'Terminology_Notes': 26, 'Relationships': 705, 'Relevant_Properties': 15, 'Observed_Examples': 357, 'Functional_Areas': 27, 'Applicable_Platforms': 556, 'Related_Attack_Patterns': 206, 'Affected_Resources': 50, 'Likelihood_of_Exploit': 184, 'Alternate_Terms': 65, 'References': 281, 'White_Box_Definitions': 29, 'Demonstrative_Examples': 385, 'Theoretical_Notes': 26, 'Enabling_Factors_for_Exploitation': 22, 'Potential_Mitigations': 523, 'Relationship_Notes': 122, 'Background_Details': 41}


Next, we plot the histogram: 

In [36]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import Range1d
from bokeh.io import output_notebook
output_notebook() #So plots

data = {}
data['Entries'] = histogram

df_data = pd.DataFrame(data).sort_values(by='Entries', ascending=True)
series = df_data.loc[:,'Entries']

p = figure(width=800, y_range=series.index.tolist(), title="Weaknesses Histogram")

p.xaxis.axis_label = 'Frequency'
p.xaxis.axis_label_text_font_size = '10pt'
p.xaxis.major_label_text_font_size = '8pt'

p.yaxis.axis_label = 'Field'
p.yaxis.axis_label_text_font_size = '10pt'
p.yaxis.major_label_text_font_size = '8pt'

j = 1
for k,v in series.iteritems():
  
  #Print fields, values, orders
  #print (k,v,j) 
  p.rect(x=v/2, y=j, width=abs(v), height=0.4,
    width_units="data", height_units="data")
  j += 1

Loading BokehJS ...

In [37]:
show(p)

# Website

The data for the website is significantly more structured. There is a strict hierarchy. However, this hierarchy is not explicitly stated so it should be noted that the hierarchy presented is only a working hypothoseis.

In [38]:
from IPython.display import Image
Image(filename='CWE_Fields.png')

The hierarchy is realatively simple, a node cannot contain a node that is higher than it in the hierarchy. A node can contain itself. For example a **Weakness Base** node will never contain a **Weakness Class** node. The only exception is **Category** which can contain and be contained in both **Weakness Class** and **Weakness Base** nodes however it will never allow a **Weakness Class** to be contained in a **Weakness Base** recursively. 

# Field Analysis

Now that we have the frequencies of the fields we can determine which fields will be most useful in creating our corpus. For this purpose the arbitrary number of 100 was chosen. If a field occurs more than 100 times it will be detailed below with an example. The observations are based on limited research and analysis so they are not perfect, but they reasonably describe the rules and regulations of each field.

### Currently Useful Fields

<table>
<th>Name</th><th>Description</th><th  width="50%">Example</th>
<tr>
<td>Description</td><td>A brief description of the Weakness. Typically one to two sentances. Can also contain an Extended Description which goes into further detail. It should be noted that sometimes the extended description is shorter than the regular description but this is rarely the case</td><td><img src="files/Description.png"></td>
</tr>
<tr>
<td>Common_Consequences</td><td>A single sentence. There is also the scope which is a one or two word term as will as the technical impact which varies in format.</td><td><img src="files/Common_Consequences.png"></td> 
</tr>
<tr>
<td>Potential_Mitigations</td><td> Describes how to prevent exploit at various steps in the development cycle. Typically a single sentence per phase and there can be a variable amount of phases</td><td><img src="files/Potential_Mitigations.png"></td> 
</tr>
<tr>
<td>Demonstrative_Examples</td><td>Contains code pertaining to the weakness, but more importantly it can contain extremely detailed descriptions. The length is highly variable but the term density is high. The code is contained within an HTML tag however the notes by whoever wrote the article can be strucutred in a multitude of ways.</td><td><img src="files/Demonstrative_Example.png"><br><img src="files/Demonstrative_Example2.png"></td> 
</tr>
</table>   

### Currently Discarded Fields

**WIP**
This is where the fields that aren't currently useful will go